In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# read the charity_data.csv.
application_df = pd.read_csv("./Resources/model_charity_data.csv")
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [2]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   STATUS                        34299 non-null  int64  
 1   ASK_AMT                       34299 non-null  int64  
 2   IS_SUCCESSFUL                 34299 non-null  int64  
 3   APPLICATION_TYPE_Other        34299 non-null  float64
 4   APPLICATION_TYPE_T19          34299 non-null  float64
 5   APPLICATION_TYPE_T3           34299 non-null  float64
 6   APPLICATION_TYPE_T4           34299 non-null  float64
 7   APPLICATION_TYPE_T5           34299 non-null  float64
 8   APPLICATION_TYPE_T6           34299 non-null  float64
 9   AFFILIATION_CompanySponsored  34299 non-null  float64
 10  AFFILIATION_Family/Parent     34299 non-null  float64
 11  AFFILIATION_Independent       34299 non-null  float64
 12  AFFILIATION_National          34299 non-null  float64
 13  A

In [3]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns="IS_SUCCESSFUL")
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=78)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(25724, 40) (8575, 40) (25724,) (8575,)


In [4]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [5]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer
# NOTE: How to choose number of hidden layers/nodes: 
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
number_input_features = len(X_train_scaled[0])
hidden_layer1 = 20
hidden_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=number_input_features, activation="relu", units=hidden_layer1))

# Second hidden layer
nn.add(tf.keras.layers.Dense(input_dim=number_input_features, activation="relu", units=hidden_layer2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                820       
                                                                 
 dense_1 (Dense)             (None, 5)                 105       
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 931
Trainable params: 931
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized1/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5" # https://keras.io/api/callbacks/model_checkpoint/

In [7]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
           
# Create a callback object that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
        filepath = checkpoint_path,
        verbose=1,
        save_weights_only=True,
        save_freq=5
)

In [8]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 868us/step - loss: 0.5920 - accuracy: 0.6979
Epoch 2/100
804/804 [==============================] - 1s 853us/step - loss: 0.5635 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 1s 880us/step - loss: 0.5588 - accuracy: 0.7283
Epoch 4/100
804/804 [==============================] - 1s 929us/step - loss: 0.5570 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 1s 964us/step - loss: 0.5556 - accuracy: 0.7302
Epoch 6/100
804/804 [==============================] - 1s 918us/step - loss: 0.5546 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 1s 838us/step - loss: 0.5537 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 1s 888us/step - loss: 0.5524 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 1s 913us/step - los

804/804 [==============================] - 1s 794us/step - loss: 0.5438 - accuracy: 0.7375
Epoch 80/100
804/804 [==============================] - 1s 958us/step - loss: 0.5443 - accuracy: 0.7378
Epoch 81/100
804/804 [==============================] - 1s 876us/step - loss: 0.5440 - accuracy: 0.7375
Epoch 82/100
804/804 [==============================] - 1s 890us/step - loss: 0.5438 - accuracy: 0.7379
Epoch 83/100
804/804 [==============================] - 1s 855us/step - loss: 0.5438 - accuracy: 0.7376
Epoch 84/100
804/804 [==============================] - 1s 861us/step - loss: 0.5436 - accuracy: 0.7379
Epoch 85/100
804/804 [==============================] - 1s 833us/step - loss: 0.5440 - accuracy: 0.7373
Epoch 86/100
804/804 [==============================] - 1s 837us/step - loss: 0.5439 - accuracy: 0.7374
Epoch 87/100
804/804 [==============================] - 1s 821us/step - loss: 0.5436 - accuracy: 0.7380
Epoch 88/100
804/804 [==============================] - 1s 841us/step - loss:

In [9]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2, callbacks= [cp_callback])
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5563 - accuracy: 0.7262 - 256ms/epoch - 955us/step
Loss: 0.5562852621078491, Accuracy: 0.7261807322502136


* By adding a second hidden layer with five nodes, the accuracy marginally increased (from 72.4% to 72.6%), and the loss marginally decreased (from 0.55632 to 0.55629) but did not reach the desired accuracy of 75.0%.